#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> Dr: Alejandro Pimentel  </font>

##### <font color="#03997A"> Modulo 8 Introducción al Deep Learning </font>

##### <font color="#03997A"> Ejercicio 4 CNN  Alumno: Ibarra Ramírez Sergio </font>

In [11]:
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder

X,y = fetch_openml('mnist_784', as_frame=False, return_X_y=True)

X = normalize(X).reshape(-1,28,28,1)

X_train = X[:60000]
X_test = X[60000:]
y_train = y[:60000]
y_test = y[60000:]

codificador = OneHotEncoder(handle_unknown='ignore',sparse=False)
y_train = codificador.fit_transform(y_train.reshape(-1,1))
y_test = codificador.transform(y_test.reshape(-1,1))

X_train = tf.constant(X_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
batch_size = 100 # Hiperparámetro
train_dataset = train_dataset.shuffle(batch_size).batch(batch_size)

c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
# Initialize weights
b = tf.Variable(tf.zeros([10]))
W = tf.Variable(tf.random.truncated_normal([1568, 10], stddev=0.1)) 

conv1_filter = tf.Variable(tf.random.truncated_normal([4, 4, 1, 16], stddev=0.1))
conv1_bias =  tf.Variable(tf.random.truncated_normal([16], stddev=0.1))

conv2_filter = tf.Variable(tf.random.truncated_normal([4, 4, 16, 32], stddev=0.1))
conv2_bias = tf.Variable(tf.random.truncated_normal([32], stddev=0.1))

# Training loop
optimizador = tf.keras.optimizers.experimental.SGD(learning_rate=0.005)


epochs = 10

for epoch in range(epochs):
  loss_epoch = 0
  for x, y in train_dataset:
    with tf.GradientTape() as gt:
    
      conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,2,2,1], padding='SAME')
      conv1_mas_bias = tf.nn.bias_add(conv1, conv1_bias)
      conv1_salida_activacion = tf.nn.relu(conv1_mas_bias)
      
      conv2 = tf.nn.conv2d(conv1_salida_activacion, conv2_filter, strides=[1,2,2,1], padding='SAME')  
      conv2_mas_bias = tf.nn.bias_add(conv2, conv2_bias)
      conv2_salida_activacion = tf.nn.relu(conv2_mas_bias)

      flat = tf.reshape(conv2_salida_activacion, [-1, 1568])
      
      dense = tf.matmul(flat, W) + b  
      
      
      x_ent = tf.nn.softmax_cross_entropy_with_logits(
          logits = dense,
          labels = y
      )
      
      loss = tf.reduce_mean(x_ent)
      optimizador.minimize(loss,[W,b,conv1_filter,b_conv1, conv2_filter, b_conv2],gt) # Poderosa

    loss_epoch += loss/len(train_dataset)
  print(loss_epoch)

tf.Tensor(2.3039744, shape=(), dtype=float32)
tf.Tensor(2.2832217, shape=(), dtype=float32)
tf.Tensor(2.257505, shape=(), dtype=float32)
tf.Tensor(2.1964169, shape=(), dtype=float32)
tf.Tensor(2.0284784, shape=(), dtype=float32)
tf.Tensor(1.5714421, shape=(), dtype=float32)
tf.Tensor(0.9279587, shape=(), dtype=float32)
tf.Tensor(0.6191782, shape=(), dtype=float32)
tf.Tensor(0.5079457, shape=(), dtype=float32)
tf.Tensor(0.45420605, shape=(), dtype=float32)


In [14]:
conv1 = tf.nn.conv2d(X_test, conv1_filter, strides=[1,2,2,1], padding='SAME')
conv1_mas_bias = tf.nn.bias_add(conv1, conv1_bias)
conv1_salida_activacion = tf.nn.relu(conv1_mas_bias)
      
conv2 = tf.nn.conv2d(conv1_salida_activacion, conv2_filter, strides=[1,2,2,1], padding='SAME')  
conv2_mas_bias = tf.nn.bias_add(conv2, conv2_bias)
conv2_salida_activacion = tf.nn.relu(conv2_mas_bias)

flat = tf.reshape(conv2_salida_activacion, [-1, 1568])
      
dense = tf.matmul(flat, W) + b  


probs = tf.nn.softmax(dense)

max_preds = tf.argmax(probs,1)
max_trues = tf.argmax(y_test,1)

numCorrectos = tf.equal(max_preds,max_trues)
acc = tf.reduce_mean(tf.cast(numCorrectos,tf.float32)) # Tengo que convertirlo porque el original es booleano
print(acc)
      
      
      

tf.Tensor(0.883, shape=(), dtype=float32)
